# <a id='toc1_'></a>[Data Preprocessing (PREP)](#toc0_)

**Objective:** Clean and preprocess the data to ensure it is standardized, free of errors, and ready for integration.

---

**Table of contents**<a id='toc0_'></a>    
- [Data Preprocessing (PREP)](#toc1_)    
  - [Parameters](#toc1_1_)    
  - [Imports](#toc1_2_)    
  - [Load input data](#toc1_3_)    
  - [Counting unique values to identify classes](#toc1_4_)    
  - [Check for deleted samples](#toc1_5_)    
  - [Add `URL` colum to the data](#toc1_6_)    
  - [Show number of samples by `sample_name`](#toc1_7_)    
  - [Exclude samples with issues](#toc1_8_)    
  - [Find and check samples with `sample_name`==`unknown` or empty](#toc1_9_)    
  - [Checking if there are any missing and unknown values in the `processed_file_location` column](#toc1_10_)    
- [Ckeck the urls for the `processed_file_location` column](#toc2_)    
    - [Add colum n image_name if doesn't exist](#toc2_1_1_)    
  - [Hash](#toc2_2_)    
  - [Save Cleaned Data](#toc2_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

---


## <a id='toc1_1_'></a>[Parameters](#toc0_)

In [1]:
DATASET_NAME = 'MHS-Tanzania'
INPUT_DATA_FNAME = "data/processed/recovered_data.csv"
OUTPUT_DATA_FNAME = "data/processed/MSH-Tanzania_metadata_all.csv"
TMP_IMG_DIR = 'data/intermediate/images/' 

## <a id='toc1_2_'></a>[Imports](#toc0_)

In [2]:
import pandas as pd
import sys

sys.path.append("../../src/")
from utils import *


## <a id='toc1_3_'></a>[Load input data](#toc0_)

In [3]:
initial_data = pd.read_csv(INPUT_DATA_FNAME)
print(f"Samples: {len(initial_data)}")
print(f"Columns: {len(initial_data.columns)}")
print(f"\t{(initial_data.columns)}")

Samples: 2949
Columns: 27
	Index(['id', 'sample_name', 'test_name', 'user_name', 'date_of_creation',
       'raw_file_location', 'processed_file_location', 'processing_date',
       'camera_type_1', 'notes', 'sample_id', 'quantity', 'deleted', 'issue',
       'project.id', 'project.user_name', 'project.project_name',
       'project.annotation', 'project.test_name',
       'project.sample_names.sample_names', 'project.neutral_filler',
       'project.qpc20', 'project.qpc50', 'project.qpc80', 'project.qpc100',
       'project.notes', 'test_id'],
      dtype='object')


In [4]:
initial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2949 entries, 0 to 2948
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 2949 non-null   int64  
 1   sample_name                        2949 non-null   object 
 2   test_name                          2949 non-null   object 
 3   user_name                          2949 non-null   object 
 4   date_of_creation                   2949 non-null   object 
 5   raw_file_location                  2944 non-null   object 
 6   processed_file_location            2949 non-null   object 
 7   processing_date                    2949 non-null   object 
 8   camera_type_1                      2949 non-null   object 
 9   notes                              2949 non-null   object 
 10  sample_id                          2949 non-null   int64  
 11  quantity                           2949 non-null   int64

## <a id='toc1_4_'></a>[Counting unique values to identify classes](#toc0_)

In [5]:
num_classes = initial_data.sample_name.nunique()
print(f"Number of classes: {num_classes}")

display(initial_data.sample_name.value_counts())

Number of classes: 10


sample_name
vitamin-c              636
paracetamol            326
benzyl-penicillin      317
penicillin-procaine    310
quinine                298
amoxicillin            296
lactose                225
cellulose              215
starch                 214
paracetamol-starch     112
Name: count, dtype: int64

## <a id='toc1_5_'></a>[Check for deleted samples](#toc0_)

In [6]:
# Filter by deleted
num_cards = len(initial_data.index)
initial_data = initial_data[~initial_data['deleted']]

print(f"Deleted cards: {num_cards - len(initial_data.index)}")

Deleted cards: 0


## <a id='toc1_6_'></a>[Add `URL` colum to the data](#toc0_)

In [7]:
# Add url to dataframe
initial_data['url'] = initial_data['processed_file_location'].apply(lambda x: f"https://pad.crc.nd.edu/{x}")

### <a id='toc2_1_1_'></a>[Add column `image_name`](#toc0_)

In [8]:
initial_data['image_name'] = initial_data.apply(lambda x: create_filename(x), axis=1)


## <a id='toc1_7_'></a>[Show number of samples by `sample_name`](#toc0_)

In [9]:
initial_data.value_counts(['sample_name']).reset_index(name='counts')

,sample_name,counts
0,vitamin-c,636
1,paracetamol,326
2,benzyl-penicillin,317
3,penicillin-procaine,310
4,quinine,298
5,amoxicillin,296
6,lactose,225
7,cellulose,215
8,starch,214
9,paracetamol-starch,112


## <a id='toc1_8_'></a>[Exclude samples with issues](#toc0_)

In [10]:
# select cards that have no issues
size_before = len(initial_data.index)
initial_data = initial_data[initial_data['issue'].isnull()].copy()
size_after = len(initial_data.index)

print(f"Samples with issues: {size_before-size_after} samples")

Samples with issues: 0 samples


## <a id='toc1_9_'></a>[Find and check samples with `sample_name`==`unknown` or empty](#toc0_)

In [11]:
column_name = "sample_name"

print(f"Total samples: {len(initial_data.index)}")

empty_name = filter_by_empty_column(initial_data, column_name)
print(f"Total num of samples with empty {column_name}: {len(empty_name.index)} samples")

unknown_name = filter_by_unknown_column(initial_data, column_name)
print(f"Total num of samples with unknown sample_name: {len(unknown_name.index)} samples")

Total samples: 2949
Total num of samples with empty sample_name: 0 samples
Total num of samples with unknown sample_name: 0 samples


## <a id='toc1_10_'></a>[Checking if there are any missing and unknown values in the `processed_file_location` column](#toc0_)

In [12]:
column_name = "processed_file_location"

empty_name = filter_by_empty_column(initial_data, column_name)
print(f"Total num of samples with empty '{column_name}': {len(empty_name.index)} samples")

unknown_name = filter_by_unknown_column(initial_data, column_name)
print(f"Total num of samples with unknown '{column_name}': {len(unknown_name.index)} samples")

Total num of samples with empty 'processed_file_location': 0 samples
Total num of samples with unknown 'processed_file_location': 0 samples


# <a id='toc2_'></a>[Ckeck the urls for the `processed_file_location` column](#toc0_)

In [13]:
# column_name = "url"

# bad_urls_df = check_url(initial_data)
# print(f"Samples with bad urls: {len(bad_urls_df.index)} samples")

# # save the samples that have a status code different from 200 in a new csv file called check_samples_with_bad_urls.csv
# if len(bad_urls_df.index) > 0:
#     bad_urls_df.to_csv('../data/intermediate/FHI2020_analysis/check_samples_with_bad_urls.csv', index=False)

## <a id='toc2_2_'></a>[Hash](#toc0_)

- Downloading images and Calculating the hash of the processed files

In [14]:
def get_url_status_code(url):
    r = requests.head(url, verify=False)
    return r.status_code

def get_hash(sample, folder):
    filename = create_filename(sample)
    filepath = os.path.join(folder, filename)
    #print(sample['id'], sample['url_status_code'], filepath)
    if sample['url_status_code'] == 200:
        if os.path.isfile(filepath):
            hashlib_md5 = hashlib.md5(open(filepath,'rb').read()).hexdigest()
            #print(hashlib_md5)
            return hashlib_md5
        else:
            print(f"File not found {filepath}. Downloading...")
            status_code = save_image_from_url(sample, folder)
            if status_code == 200:
                hashlib_md5 = hashlib.md5(open(filepath,'rb').read()).hexdigest()
                return hashlib_md5
            else:
                print(f"Error downloading {sample['id']}. File not found {filepath}")
                return None
    else:        
        return None

def save_image_from_url(sample, output_folder):
    r = requests.get(sample.url, verify=False)
    output_path = os.path.join(
        output_folder, create_filename(sample)
    )
    with open(output_path, "wb") as f:
        f.write(r.content)
    
    # Fix image if needed
    img = Image.open(output_path)
    try:
        
        img.save(output_path)
        img = Image.open(output_path)
        img.verify()
    except Exception as exc:
        print(f"Image {output_path} is broken")
    
    status_code = r.status_code
    r.close()
    return status_code

def get_hash_all(df, folder):    
    column_exists = "url_status_code" in df.columns
    
    if ~column_exists:
        df['url_status_code'] = df['url'].apply(lambda x: get_url_status_code(x))
    
    hash_codes = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {
            executor.submit(get_hash, sample, folder): sample for _, sample in df.iterrows()
        }
        for future in concurrent.futures.as_completed(
            future_to_url
        ):
            sample = future_to_url[future]
            try:               
                #print(sample['id'], sample['url'], sample['url_status_code'] ,future.result())
                hash_codes.append([sample['id'], sample['url_status_code'], future.result()])
                
            except Exception as exc:
                print(
                    "%s generated an exception: %s" % (sample['id'], exc)
                )
    return hash_codes

In [15]:
hash_codes = get_hash_all(initial_data, TMP_IMG_DIR)

In [16]:
hash_codes_df = pd.DataFrame(hash_codes, columns=['id', 'url_status_code', 'hashlib_md5'])
hash_codes_df.url_status_code.unique()

array([200])

- check is all images have hash 

In [17]:
no_hash = hash_codes_df[hash_codes_df.hashlib_md5.isnull()]
print(f"Samples with null hash: {len(no_hash)}")

no_image = hash_codes_df[hash_codes_df.url_status_code != 200]
print(f"Samples with no image: {len(no_image)}")

Samples with null hash: 0
Samples with no image: 0


- Finally, you can create a new column called `hashlib_md5` with the hash of the images

In [18]:
# drop 'url_status_code' column from dataframes
# hash_codes_df.drop(columns=['url_status_code'], inplace=True)
#if 'url_status_code' in df.columns: df.drop(columns=['url_status_code'], inplace=True)

# merge for adding the 'hashlib_md5' column
initial_data = pd.merge(initial_data, hash_codes_df, on='id')

initial_data.columns

Index(['id', 'sample_name', 'test_name', 'user_name', 'date_of_creation',
       'raw_file_location', 'processed_file_location', 'processing_date',
       'camera_type_1', 'notes', 'sample_id', 'quantity', 'deleted', 'issue',
       'project.id', 'project.user_name', 'project.project_name',
       'project.annotation', 'project.test_name',
       'project.sample_names.sample_names', 'project.neutral_filler',
       'project.qpc20', 'project.qpc50', 'project.qpc80', 'project.qpc100',
       'project.notes', 'test_id', 'url', 'image_name', 'url_status_code_x',
       'url_status_code_y', 'hashlib_md5'],
      dtype='object')

- check if there are any samples that have the same hash

In [19]:
num_samples = len(initial_data)
data = initial_data.groupby(['hashlib_md5']).size().reset_index(name='counts')
one_sample_hash = data[data['counts']==1]
two_more_sample_hash = data[data['counts']>1]

print('Summary:')
print(f"Total unique hash codes : {len(data.index)}")
print(f"Total of hash code with one sample: {len(one_sample_hash.index)}")
print(f"Total of hash code with two or more samples: {len(two_more_sample_hash.index)}")

print('')
print(f"Total of samples: {num_samples}")
print(f"Total of samples without duplicates: {len(data.index)}")
print(f"Total of samples in some duplicate case (will be deleted): {num_samples-len(data.index)}")

Summary:
Total unique hash codes : 2949
Total of hash code with one sample: 2949
Total of hash code with two or more samples: 0

Total of samples: 2949
Total of samples without duplicates: 2949
Total of samples in some duplicate case (will be deleted): 0


- Sort by `sample_name` 

In [20]:
sorted_data = initial_data.sort_values(by=['sample_name', 'id', 'sample_id'], ascending=[True, False, False])
sorted_data_reset = sorted_data.reset_index(drop=True)

## <a id='toc2_3_'></a>[Save Cleaned Data](#toc0_)

On that point the dataframe `data` should have the cleaned data samples to put in the dataset.

In [21]:
# save cleaned dataframe to csv
data = sorted_data[['id','sample_id','sample_name', 'quantity', 'project.id', 'url', 'hashlib_md5', 'image_name']]
data.to_csv(OUTPUT_DATA_FNAME, index=False)